In [2]:
from halide import *
min_, max_ = __builtins__.min, __builtins__.max

In [3]:
x, y = Var("x"), Var("y")

In [3]:
#default ordering
gradient = Func("gradient")
gradient[x,y] = x + y
gradient.trace_stores()
print("Evaluating gradient row-major")
output = gradient.realize(4,4)

#The equivalent C is
for yy in range(4):
    for xx in range(4):
        print("Evaluating at x = %d, y = %d: %d" % (xx, yy, xx + yy))
print("\n")

print("Pseudo-code for schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient row-major
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6


Pseudo-code for schedule:



In [4]:
#Reorder variables
gradient = Func("gradient_col_major")
print("x, y",x, y)
gradient[x,y] = x + y
gradient.trace_stores()

gradient.reorder(y,x)
print("Evaluating gradient column-major")
output = gradient.realize(4,4)

print("Evaluatin C:")
for xx in range(4):
    for yy in range(4):
        print("Evaluating at x = %d, y = %d, %d" % (xx,yy,xx+yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

x, y <halide.Var 'x'> <halide.Var 'y'>
Evaluating gradient column-major
Evaluatin C:
Evaluating at x = 0, y = 0, 0
Evaluating at x = 0, y = 1, 1
Evaluating at x = 0, y = 2, 2
Evaluating at x = 0, y = 3, 3
Evaluating at x = 1, y = 0, 1
Evaluating at x = 1, y = 1, 2
Evaluating at x = 1, y = 2, 3
Evaluating at x = 1, y = 3, 4
Evaluating at x = 2, y = 0, 2
Evaluating at x = 2, y = 1, 3
Evaluating at x = 2, y = 2, 4
Evaluating at x = 2, y = 3, 5
Evaluating at x = 3, y = 0, 3
Evaluating at x = 3, y = 1, 4
Evaluating at x = 3, y = 2, 5
Evaluating at x = 3, y = 3, 6

Pseudo-code for the schedule:



In [5]:
#Split a variable into two
gradient = Func("gradient_split")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, x_inner = Var("x_outer"), Var("x_inner")
gradient.split(x, x_outer, x_inner, 2)

print("Evaluating gradient with x split into x_outer and x_inner")
output = gradient.realize(4,4)

print("Equivalent C:")
for yy in range(4):
    for x_outer in range(2):
        for x_inner in range(2):
            xx = x_outer * 2 + x_inner
            print("Evaluating at x = %d, y = %d: %d" % (xx, yy, xx + yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient with x split into x_outer and x_inner
Equivalent C:
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6

Pseudo-code for the schedule:



In [6]:
# Fuse two variables into one
gradient = Func("Gradient_fused")
gradient[x,y] = x + y

fused = Var("fused")
gradient.fuse(x, y, fused)

print("Evaluating gradient with x and y fused")
output = gradient.realize(4,4)

print("Equivalent C:")
for fused in range(4*4):
    yy = fused / 4
    xx = fused % 4
    print("Evaluating at x = %d, y = %d: %d" % (xx, yy, xx + yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient with x and y fused
Equivalent C:
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6

Pseudo-code for the schedule:



In [7]:
# Evaluating in tiles
gradient = Func("gradient_tiled")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, x_inner, y_outer, y_inner = Var(), Var(), Var(), Var()
gradient.split(x, x_outer, x_inner, 2)
gradient.split(y, y_outer, y_inner, 2)
gradient.reorder(x_inner, y_inner, x_outer, y_outer)

print("Evaluating gradient in 2x2 tiles")
output = gradient.realize(4, 4)

print("Equivalent C:")
for y_outer in range(2):
    for x_outer in range(2):
        for y_inner in range(2):
            for x_inner in range(2):
                xx = x_outer * 2 + x_inner
                yy = y_outer * 2 + y_inner
                print("Evaluating at x = %d, y = %d: %d" % (xx, yy, xx + yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient in 2x2 tiles
Equivalent C:
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6

Pseudo-code for the schedule:



In [8]:
# Evaluating vectors
gradient = Func("gradient_in_vectors")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, x_inner = Var("x_outer"), Var("x_inner")
gradient.split(x, x_outer, x_inner, 4)
gradient.vectorize(x_inner)

print("Evaluating gradient with x_inner vectorized")
output = gradient.realize(8,4)

print("Equivalent C:")
for yy in range(4):
    for x_outer in range(2):
        x_vec = [x_outer * 4 + 0,
                 x_outer * 4 + 1,
                 x_outer * 4 + 2,
                 x_outer * 4 + 3]
        val = [x_vec[0] + yy,
               x_vec[1] + yy,
               x_vec[2] + yy,
               x_vec[3] + yy]
        print("Evaluating at <%d, %d, %d, %d> <%d, %d, %d, %d>: <%d, %d, %d, %d>" % (
            x_vec[0],x_vec[1],x_vec[2],x_vec[3],
            yy, yy, yy, yy,
            val[0], val[1], val[2], val[3])) 
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient with x_inner vectorized
Equivalent C:
Evaluating at <0, 1, 2, 3> <0, 0, 0, 0>: <0, 1, 2, 3>
Evaluating at <4, 5, 6, 7> <0, 0, 0, 0>: <4, 5, 6, 7>
Evaluating at <0, 1, 2, 3> <1, 1, 1, 1>: <1, 2, 3, 4>
Evaluating at <4, 5, 6, 7> <1, 1, 1, 1>: <5, 6, 7, 8>
Evaluating at <0, 1, 2, 3> <2, 2, 2, 2>: <2, 3, 4, 5>
Evaluating at <4, 5, 6, 7> <2, 2, 2, 2>: <6, 7, 8, 9>
Evaluating at <0, 1, 2, 3> <3, 3, 3, 3>: <3, 4, 5, 6>
Evaluating at <4, 5, 6, 7> <3, 3, 3, 3>: <7, 8, 9, 10>

Pseudo-code for the schedule:



In [9]:
# Unrolling a loop
gradient = Func("gradient_in_vectors")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, x_inner = Var("x_outer"), Var("x_inner")
gradient.split(x, x_outer, x_inner, 2)
gradient.unroll(x_inner)

print("Evaluating gradient unrolled by a factor of two")
result = gradient.realize(4,4)

print("Evaluating C:")
for yy in range(4):
    for x_outer in range(2):
        x_inner = 0
        xx = x_outer * 2 + x_inner
        print("Evaluating at x = %d, y = %d: %d" % (xx, yy, (xx + yy)))
        
        x_inner = 1
        xx = x_outer * 2 + x_inner
        print("Evaluating at x = %d, y = %d: %d" % (xx, yy, (xx + yy)))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient unrolled by a factor of two
Evaluating C:
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6

Pseudo-code for the schedule:



In [10]:
# Splitting by factors that don't divide the extent
gradient = Func("gradient_split_5x4")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, x_inner = Var("x_outer"), Var("x_inner")
gradient.split(x, x_outer, x_inner, 2)

print("Evaluating gradient over a 5x4 box with x split by two")
output = gradient.realize(5,4)

print("Equaivalent C:")
for yy in range(4):
    for x_outer in range(3):
        for x_inner in range(2):
            xx = x_outer * 2
            if xx > 3:
                xx = 3
            xx += x_inner
            print("Evaluating at x = %d, y = %d: %d" % (xx,yy,xx+yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient over a 5x4 box with x split by two
Equaivalent C:
Evaluating at x = 0, y = 0: 0
Evaluating at x = 1, y = 0: 1
Evaluating at x = 2, y = 0: 2
Evaluating at x = 3, y = 0: 3
Evaluating at x = 3, y = 0: 3
Evaluating at x = 4, y = 0: 4
Evaluating at x = 0, y = 1: 1
Evaluating at x = 1, y = 1: 2
Evaluating at x = 2, y = 1: 3
Evaluating at x = 3, y = 1: 4
Evaluating at x = 3, y = 1: 4
Evaluating at x = 4, y = 1: 5
Evaluating at x = 0, y = 2: 2
Evaluating at x = 1, y = 2: 3
Evaluating at x = 2, y = 2: 4
Evaluating at x = 3, y = 2: 5
Evaluating at x = 3, y = 2: 5
Evaluating at x = 4, y = 2: 6
Evaluating at x = 0, y = 3: 3
Evaluating at x = 1, y = 3: 4
Evaluating at x = 2, y = 3: 5
Evaluating at x = 3, y = 3: 6
Evaluating at x = 3, y = 3: 6
Evaluating at x = 4, y = 3: 7

Pseudo-code for the schedule:



In [11]:
# Fusing, tiling and parallelizing
gradient = Func("gradinet_fused_tiles")
gradient[x,y] = x + y
gradient.trace_stores()

x_outer, y_outer = Var("x_outer"), Var("y_outer")
x_inner, y_inner = Var("x_inner"), Var("y_inner")
tile_index = Var("tile_index")
gradient.tile(x,y,x_outer,y_outer,x_inner,y_inner,2,2).fuse(x_outer, y_outer,tile_index).parallel(tile_index)

print("Evaluating gradient tiles in parallel")
output = gradient.realize(4,4)

print("Equivalent (serial) C:")
for tile_index in range(4):
    y_outer = tile_index / 2
    x_outer = tile_index % 2
    for y_inner in range(2):
        for x_inner in range(2):
            yy = y_outer * 2 + y_inner
            xx = x_outer * 2 + x_inner
            print("Evaluating at x = %d, y = %d:%d" % (xx, yy, xx + yy))
print()

print("Pseudo-code for the schedule:")
gradient.print_loop_nest()
print()

Evaluating gradient tiles in parallel
Equivalent (serial) C:
Evaluating at x = 0, y = 0:0
Evaluating at x = 1, y = 0:1
Evaluating at x = 0, y = 1:1
Evaluating at x = 1, y = 1:2
Evaluating at x = 2, y = 1:3
Evaluating at x = 3, y = 1:4
Evaluating at x = 2, y = 2:4
Evaluating at x = 3, y = 2:5
Evaluating at x = 0, y = 2:2
Evaluating at x = 1, y = 2:3
Evaluating at x = 0, y = 3:3
Evaluating at x = 1, y = 3:4
Evaluating at x = 2, y = 3:5
Evaluating at x = 3, y = 3:6
Evaluating at x = 2, y = 4:6
Evaluating at x = 3, y = 4:7

Pseudo-code for the schedule:



In [1]:
# Putting it all together
gradient_fast = Func("gradient_fast")
gradient[x,y] = x + y

x_outer, y_outer = Var("x_outer"), Var("y_outer")
x_inner, y_inner = Var("x_inner"), Var("y_inner")
tile_index = Var("tile_index")
gradient_fast \
    .tile(x, y, x_outer, y_outer, x_inner, y_inner, 256, 256) \
    .fuse(x_outer, y_outer, tile_index) \
    .parallel(tile_index)

x_inner_outer, y_inner_outer = Var("x_inner_outer"), Var("y_inner_outer")
x_vectors, y_pairs = Var("x_vectors"), Var("y_pairs")
gradient_fast \
    .tile(x_inner, y_inner, x_inner_outer, y_inner_outer, x_vectors, y_pairs, 4, 2 )\
    .vectorize(x_vectors) \
    .unroll(y_pairs)

#result = gradient_fast.realize(800,600)
result = gradient_fast.realize(256,256)

#print("Checking Halide result against equivalent C...")
#for tile_index in range(4 * 3):
#    y_outer = tile_index // 4
#    x_outer = tile_index % 4
#    for y_inner_outer in range(256 // 2):
#        for x_inner_outer in range(256  // 4):
#            xx = min_(x_outer * 256, 800 - 256) + x_inner_outer * 4
#            x_vec = [xx + 0, xx + 1, xx + 2, xx + 3]
#            y_base = min_(y_outer * 256, 600 - 256) + y_inner_outer * 2
#            
#            yy = y_base + 0
#            y_vec = [yy, yy, yy, yy]
#            val = [x_vec[0] + y_vec[0],
#                   x_vec[1] + y_vec[1],
#                   x_vec[2] + y_vec[2],
#                   x_vec[3] + y_vec[3]]
#            for i in range(4):
#                if result(x_vec[i],y_vec[i]) != val[i]:
#                    print("There was an error at %d %d!" % (x_vac[i], y_vec[i]))
#            
#            yy = y_base + 1
#            y_vec = [yy, yy, yy, yy]
#            val = [x_vec[0] + y_vec[0],
#                   x_vec[1] + y_vec[1],
#                   x_vec[2] + y_vec[2],
#                   x_vec[3] + y_vec[3]]
#            for i in range(4):
#                if result(x_vec[i],y_vec[i]) != val[i]:
#                    print("There was an error at %d %d!" % (x_vac[i], y_vec[i]))
#print()

print("Pseudo-code for the schedule:")
gradient_fast.print_loop_nest()
print()
            

NameError: name 'Func' is not defined